<a href="https://colab.research.google.com/github/sanhiitaa/100-days-of-deep-learning/blob/main/age_gender_non_linear_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing the kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
# dowloading the dataset
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
 97% 321M/331M [00:04<00:00, 90.6MB/s]
100% 331M/331M [00:04<00:00, 85.9MB/s]


In [3]:
# unzipping the file
import zipfile
zip_ref=zipfile.ZipFile('/content/utkface-new.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
# importing libraries
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [5]:
folder_path='/content/utkface_aligned_cropped/UTKFace'

In [6]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [7]:
df=pd.DataFrame({'age': age,
                 'gender': gender,
                 'image': img_path})
df.head()

,age,gender,image
0,1,0,1_0_2_20161219190003947.jpg.chip.jpg
1,25,0,25_0_3_20170104214602149.jpg.chip.jpg
2,26,0,26_0_2_20170104023216134.jpg.chip.jpg
3,38,1,38_1_1_20170113012502193.jpg.chip.jpg
4,29,1,29_1_2_20170105164315483.jpg.chip.jpg


In [8]:
df.shape

(23708, 3)

In [9]:
train_df=df.sample(frac=1).iloc[:20000]
test_df=df.sample(frac=1).iloc[20000:]

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.3,
                                   shear_range=0.4,
                                   zoom_range=0.1,
                                   horizontal_flip=True)

train_generator= train_datagen.flow_from_dataframe(train_df,
                                                   directory=folder_path,
                                                   x_col='image',
                                                   y_col=['age', 'gender'],
                                                   target_size=(200,200),
                                                   class_mode='multi_output')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator= test_datagen.flow_from_dataframe(test_df,
                                                   directory=folder_path,
                                                   x_col='image',
                                                   y_col=['age', 'gender'],
                                                   target_size=(200,200),
                                                   class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [11]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [12]:
resnet= ResNet50(include_top=False,
                 input_shape=(200,200,3))

resnet.trainable=False

# setting the last layer of resnet conv_base as output layer
conv_output = resnet.layers[-1].output

# joining the output layer with flatten
flatten= Flatten()(conv_output)

# joining hidden layers to the flatten layer
dense1= Dense(512, activation='relu')(flatten)
dense2= Dense(512, activation='relu')(flatten)

dense3= Dense(512, activation='relu')(dense1)
dense4= Dense(512, activation='relu')(dense2)

output1= Dense(1, activation='linear', name='age')(dense3)
output2= Dense(1, activation='sigmoid', name='gender')(dense4)

94765736/94765736 [==============================] - 1s 0us/step


In [13]:
model=Model(inputs=resnet.input, outputs=[output1, output2])

In [14]:
model.compile(optimizer='adam',
              loss={'age': 'mae',
                    'gender': 'binary_crossentropy'},
              metrics={'age': 'mae',
                    'gender': 'accuracy'},
              loss_weights={'age':1,
                           'gender':99})

In [15]:
model.fit(train_generator,
          batch_size=32,
          epochs=10,
          validation_data=test_generator)

Epoch 1/10
625/625 [==============================] - 271s 417ms/step - loss: 94.2570 - age_loss: 15.4197 - gender_loss: 0.7963 - age_mae: 15.4197 - gender_accuracy: 0.5175 - val_loss: 83.3049 - val_age_loss: 14.7240 - val_gender_loss: 0.6927 - val_age_mae: 14.7240 - val_gender_accuracy: 0.5151
Epoch 2/10
625/625 [==============================] - 270s 432ms/step - loss: 83.5600 - age_loss: 15.0142 - gender_loss: 0.6924 - age_mae: 15.0142 - gender_accuracy: 0.5239 - val_loss: 83.2833 - val_age_loss: 14.6254 - val_gender_loss: 0.6935 - val_age_mae: 14.6254 - val_gender_accuracy: 0.5151
Epoch 3/10
625/625 [==============================] - 261s 417ms/step - loss: 83.7354 - age_loss: 14.9742 - gender_loss: 0.6946 - age_mae: 14.9742 - gender_accuracy: 0.5236 - val_loss: 83.1546 - val_age_loss: 14.5481 - val_gender_loss: 0.6930 - val_age_mae: 14.5481 - val_gender_accuracy: 0.5151
Epoch 4/10
625/625 [==============================] - 253s 404ms/step - loss: 83.3938 - age_loss: 14.8168 - gend